In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

In [ ]:
uploaded = files.upload()
#spot_by_celltype

Saving spot_by_celltype_4band_2reg_view1_3.csv to spot_by_celltype_4band_2reg_view1_3.csv


In [ ]:
uploaded = files.upload()
#spot_by_gene_lognorm

Saving spot_by_gene_lognorm_4band_2reg_view1_3.csv to spot_by_gene_lognorm_4band_2reg_view1_3.csv


In [ ]:
y = pd.read_csv('spot_by_celltype_4band_2reg_view1_3.csv')
X = pd.read_csv('spot_by_gene_lognorm_4band_2reg_view1_3.csv')
Xy = pd.concat([X, y], axis=1)

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler

def reg(x, y, dx, coefs=False):
    y = np.asarray(y).ravel()

    vals = np.unique(y[~np.isnan(y)])
    y_is_binary = (vals.size == 2) and np.all(np.isin(vals, [0, 1]))

    scaler = StandardScaler()
    x2  = scaler.fit_transform(x)
    dx2 = scaler.transform(dx)

    if y_is_binary:
        reg_model = LogisticRegression(random_state=0, solver="lbfgs", max_iter=2000)  # CHANGED
    else:
        reg_model = LinearRegression()

    fitted_model = reg_model.fit(x2, y)

    if y_is_binary:
        predictions = fitted_model.predict_proba(dx2)[:, 1]
    else:
        predictions = fitted_model.predict(dx2)

    predictions = np.asarray(predictions).ravel()

    return (predictions, fitted_model.coef_.ravel()) if coefs else predictions

In [ ]:
def save_ensembl_ids(genes, fname='ensembl_id_transfer.csv'):
    import requests
    import time

    server = 'https://rest.ensembl.org'
    endpoint_template = '/xrefs/symbol/homo_sapiens/{}?external_db=Ensembl_protein_id'

    rows = []
    for gene in genes:
        url = server + endpoint_template.format(gene)
        headers = {'Content-Type': 'application/json'}
        r = requests.get(url, headers=headers)
        if not r.ok:
            print(f'Failed for {gene}: {r.status_code}')
            continue
        data = r.json()
        for entry in data:
            rows.append({
                'Protein stable ID': entry['id'],
                'Gene_name': gene
            })
        time.sleep(0.1)

    df = pd.DataFrame(rows).drop_duplicates()
    df.to_csv(fname, index=False)

In [ ]:
result_adj = np.zeros([Xy.shape[1]] * 2)
init_ref_net = []
kf = KFold(n_splits=5, shuffle=True, random_state=1)

In [ ]:
for target_idx, target_label in enumerate(tqdm(Xy.columns)):
    X1 = Xy.drop(target_label, axis=1)
    Y = Xy.loc[:, target_label]

    for source_idx, source_label in enumerate(Xy.columns):
        if source_label == target_label: continue
        X2 = X1.drop(source_label, axis=1)
        if X2.to_numpy().sum() == 0: continue

        loss1_rmse = []
        loss2_rmse = []

        for train_index, test_index in kf.split(X1):
            X_train1, Y_train1 = X1.iloc[train_index], Y.iloc[train_index]
            X_test1, Y_test1 = X1.iloc[test_index], Y.iloc[test_index]
            X_train2, Y_train2 = X2.iloc[train_index], Y.iloc[train_index]
            X_test2, Y_test2 = X2.iloc[test_index], Y.iloc[test_index]

            # Regression
            Y_pre1 = reg(X_train1, Y_train1, X_test1)
            loss1 = np.sqrt(mean_squared_error(Y_test1, Y_pre1))
            loss1_rmse.append(loss1)

            Y_pre2 = reg(X_train2, Y_train2, X_test2)
            loss2 = np.sqrt(mean_squared_error(Y_test2, Y_pre2))
            loss2_rmse.append(loss2)

        mean_mse1 = np.mean(loss1_rmse)
        mean_mse2 = np.mean(loss2_rmse)

        if mean_mse2 > mean_mse1:
            result_adj[source_idx, target_idx] = 1
            init_ref_net.append({
                'Source': source_label,
                'Target': target_label,
                'E1': mean_mse1,
                'E2': mean_mse2,
                'Weight_log': np.log(mean_mse2 / mean_mse1)
            })

result_adj_df = pd.DataFrame(result_adj, index=Xy.columns, columns=Xy.columns)
result_adj_df.to_csv('Initial_ref_net_adj.csv')

init_ref_net_df = pd.DataFrame(init_ref_net)
init_ref_net_df.to_csv('Initial_ref_net.csv', index=False)

100%|██████████| 103/103 [39:19<00:00, 22.91s/it]


In [ ]:
background_network_adj = pd.read_csv('Initial_ref_net_adj.csv', index_col=0)

In [ ]:
result_network = []
for cell_idx, cell_label in [(len(Xy.index) - 1, Xy.index[-1])]: # enumerate(tqdm(Xy.index)):
    for target_idx, target_label in enumerate(Xy.columns):
        sources = np.nonzero(background_network_adj.loc[:, target_label])[0]

        x1 = Xy.drop(cell_label).iloc[:, sources]
        y = Xy.drop(cell_label).iloc[:, target_idx]

        dx1 = Xy.iloc[[cell_idx], sources]
        dy = Xy.iloc[[cell_idx], [target_idx]]

        y_pre1, y_pre1_coefs = reg(x1, y, dx1, True)
        loss1 = np.sqrt(mean_squared_error(dy, y_pre1)) + 1e-20

        for source_pos, source_idx in enumerate(sources):
            source_label = Xy.columns[source_idx]
            x2 = x1.drop(source_label, axis=1)
            dx2 = dx1.drop(source_label, axis=1)
            if x2.to_numpy().sum() == 0:
                continue

            y_pre2 = reg(x2, y, dx2)
            loss2 = np.sqrt(mean_squared_error(dy, y_pre2))

            if loss2 > loss1:
                result_network.append({
                    # 'Cell': cell_label,
                    'Source': source_label,
                    'Target': target_label,
                    'E1': loss1,
                    'E2': loss2,
                    'Weight_loss': np.log(loss2 / loss1),
                    'Source_coef': y_pre1_coefs[source_pos]
                })

result_network_df = pd.DataFrame(result_network)

In [ ]:
result_network_df["Signed_weight"] = (
    np.sign(result_network_df["Source_coef"]) * result_network_df["Weight_loss"]
)
print(result_network_df)

                              Source               Target        E1        E2  \
0                              IGFL2                HMGA2  0.118614  0.119576   
1                             S100A2                HMGA2  0.118614  0.123649   
2                               CST6                HMGA2  0.118614  0.121192   
3                             ANXA10                HMGA2  0.118614  0.120017   
4                               CHGA                HMGA2  0.118614  0.120766   
...                              ...                  ...       ...       ...   
1787                           ABCB5  Celltype_Unknown 25  0.000034  0.000036   
1788                            TCN1  Celltype_Unknown 25  0.000034  0.000074   
1789                            PAEP  Celltype_Unknown 25  0.000034  0.000140   
1790                           SOX21  Celltype_Unknown 25  0.000034  0.000034   
1791  Celltype_TFGB Activated T Cell  Celltype_Unknown 25  0.000034  0.000043   

      Weight_loss  Source_c

In [ ]:
top_n = max(1, int(len(result_network_df) * 0.05))
top_edges_df = (
    result_network_df.sort_values("Weight_loss", ascending=False)
                     .head(top_n)
                     .loc[:, ["Source", "Target", "Signed_weight"]]
)
print(top_edges_df)

                   Source                   Target  Signed_weight
597                   INS                  SLC30A8       3.313618
1612                VSIG1  Celltype_TFH CD4 T Cell       2.691952
52                   IGF1                    CALB2      -2.589315
602                  SCG3                  SLC30A8       2.508060
1660  Celltype_Unknown 25  Celltype_TFH CD4 T Cell      -2.383289
...                   ...                      ...            ...
224                 FABP4                    CILP2       0.673269
1613                 TFF1  Celltype_TFH CD4 T Cell       0.670624
509                   TTR                     CHGA       0.660616
1643                THBS4  Celltype_TFH CD4 T Cell       0.650998
1317               CRISP3                    MUC5B       0.648205

[89 rows x 3 columns]


In [ ]:
top_edges_df.to_csv("4band_2reg_view2_2_5.csv", index=False)
files.download("4band_2reg_view2_2_5.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>